# Main Work Flow
- ** Import Libraries **
- ** Data Visualization **
        - Regiospecific dependency
        - Correlations of dependency with other data types
        - Hypothesis of relation between dependency and household size
                - Large household and large dependency related to lower income level
- ** Preprocessing of training dataset **
        - Reduce the dataset to a meaningful data fields
        - Check for non-real values and change them
        - Check for data fields with more than 50% absent data and remove them
- ** Scaling **
        - Scale the data and check for zero mean and unit standard deviation
- ** Choice of method **
        - Split the training data between a training and a testing data subset
        - Apply various supervised machine learning and deep learning methods 
        - Determine the best method  based on confusion_matrix and classification_report
- ** Preprocessing of testing dataset **
        - Follow the aforementioned preprocessing steps for the training dataset
        - Avoid 'dropna' type functions to preserve predictions for all the Ids
- ** Transformation and post-processing **
        - Use of pipeline  for processing
        - Comparison of predictions with y_train dataset to gain insight about the distribution
        - Testing of hypothesis based on dependency and household size. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

# Data Import

In [ ]:
df = pd.read_csv('../input/train.csv')

** Information Check **

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
df.describe()

** Data Visualization **

In [ ]:
sns.countplot(x='Target',data=df)

In [ ]:
#
# Distribution of wealth in rural vs urban areas
#
df['areadscr'] = df['area1'].apply(lambda x: 'urban' if x==1 else 'rural') 
sns.countplot(x='Target',hue='areadscr',data=df)


**
 Distribution of wealth in various geographical areas
 **
- Unequal wealth distribution in different region 
- Major challenge in making decision on income level

In [ ]:
df['regiondscr'] = list(
map( lambda x1,x2,x3,x4,x5,x6: 
                    'Central' if x1==1 
               else 'Chorotega' if x2==1
               else 'Pacafico central' if x3==1
               else 'Brunca' if x4==1
               else 'Huetar Atlantica' if x5==1
               else 'Huetar Norte',
                    df['lugar1'],df['lugar2'],df['lugar3'],df['lugar4'],df['lugar5'],df['lugar6'] )
)
sns.factorplot(x='Target',hue='regiondscr',data=df, kind='count', log=True, size=5, aspect=1.8)

** Average dependency in different region**

In [ ]:
df_d=df.groupby(['Target','regiondscr']).mean()
fp=df_d['SQBdependency'].unstack()#.fillna(0)
plt.figure(figsize=(12,6))
sns.heatmap(fp,cmap='viridis',robust=True,annot=True)

- Cluster of region with similar dependency  
- Relation between income level and denpendency

In [ ]:
plt.figure(figsize=(12,6))
sns.clustermap(fp,cmap='viridis',annot=True)

** Inverse correlation of dependency and house hold size**
-  Data fields with weak and moderate correlation with dependency data field

In [ ]:
for i in list(df.drop(['Id','idhogar','areadscr','regiondscr','dependency','edjefe','edjefa','rez_esc'],axis=1).columns):
    x = np.corrcoef(df['SQBdependency'], df[i])[0,1]
    if abs(x) > 0.15 : 
        print( i, x )

In [ ]:
sns.factorplot(x='hogar_mayor', y='SQBdependency', data=df,kind='bar')

* ** Heat map to express  average  'income levels' over age and dependency**

In [ ]:
df_d=df.groupby(['age','SQBdependency']).mean()
fp=df_d['Target'].unstack().fillna(0)
#plt.figure(figsize=(12,6))
#ax = sns.heatmap(fp,cmap='viridis',robust=True)


yticks = fp.index
keptticks = yticks[::int(len(yticks)/6)]
yticks = ['' for y in yticks]
yticks[::int(len(yticks)/6)] = keptticks

xticks = fp.columns
keptticks = xticks[::int(len(xticks)/10)]
xticks = ['' for y in xticks]
xticks[::int(len(xticks)/10)] = keptticks

plt.figure(figsize=(12,6))
sns.heatmap(fp,linewidth=0,
            yticklabels=yticks, xticklabels=xticks,
            cmap='viridis',robust=True)

# This sets the yticks "upright" with 0, as opposed to sideways with 90.
plt.yticks(rotation=0) 

plt.show()

** Cluster of similar age groups based on  'income levels' data field and its relation with dependency**

In [ ]:
plt.figure(figsize=(12,6))
sns.clustermap(fp,cmap='viridis')

** Dependency **
- ** Households with higher dependency and larger size **

In [ ]:
np.corrcoef(df['SQBdependency'], df['hhsize'])

- Negative correlation suggests that dependency and household size are inversely proportional 

In [ ]:
g = sns.regplot(  x='hhsize',y='SQBdependency',
                  fit_reg=False, data=df )
g.set_xscale('log')
g.set_yscale('log')
plt.ylim(0.01,100)

** Aforementioned hypothesis can be tested by observing the 'income levels' data field of the training data set ** 
- Lower income levels for data sets with higher dependency and larger household size

In [ ]:
sns.lmplot(  x='hhsize',y='SQBdependency', hue='Target',
                  fit_reg=False, data=df )

# Preprocessing - (a) Value Checks

In [ ]:
df7 = df.drop(['Id','idhogar','areadscr','regiondscr','dependency'],axis=1)

In [ ]:
df7.info()

NaN values of v18q1 (number of tablets household owns) data field is replaced based on v18q (owns a tablet) data field****


In [ ]:
df7['v18q1'] = list(
                map( lambda x,y: 
                    ( int(0) if y==0 else df7['v18q1'].mean() ) if x!=x 
                      else x,
                    df7['v18q1'],df7['v18q'] )
)

'edjefe' and 'edjefe' data fields are converted form object based data fields to integer data fields

In [ ]:
df7['edjefe'] = df7['edjefe'].apply(lambda x: int(0) if x=='no'
                                         else int(1) if x=='yes'
                                         else int(x))

In [ ]:
df7['edjefa'] = df7['edjefa'].apply(lambda x: int(0) if x=='no'
                                         else int(1) if x=='yes'
                                         else int(x))

** Determine NaN values **
- these columns contains NaN values

In [ ]:
coldf=df7.columns.values.tolist()
x0 = len( df7.index )
for i in coldf:
    x = float(len( df7[pd.isnull(df7[i])].index )) 
    if x!=0 : print( i, x/x0 * 100 )

 - Data fields, which have more than 50% data unavailable, are dropped from further calculation 

In [ ]:
df8=df7.drop(['v2a1','rez_esc'],axis=1).interpolate()
#df8=df7.drop(['v2a1','rez_esc'],axis=1)
coldf=df8.columns.values.tolist()
x0 = len( df8.index )
for i in coldf:
    x = float(len( df8[pd.isnull(df8[i])].index )) 
    if x!=0 : print( i, x/x0 * 100 )

In [ ]:
df8.info()

# Preprocessing - (b) Scaling
- multivariate analysis
- zero mean with unit variance

In [ ]:
from sklearn import preprocessing

In [ ]:
X_train = df8.drop('Target',axis=1)
y_train = df8['Target']

In [ ]:
X_scaled = preprocessing.scale(X_train)

** Checking for zero mean and unit standard deviation **

In [ ]:
X_scaled.mean(axis=0)

In [ ]:
X_scaled.std(axis=0)

## Train Test Split
- The test data doesn't include y_train values 
- Test few methodologies to model the system 
- Test the accuracy of the methods

** Let's split up the data into a training set and a test set! **

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
XX_train, XX_test, yy_train, yy_test = train_test_split(X_scaled, y_train, test_size=0.30)

# Methods / Approaches
** Method 1 : Machine Learning - single decision tree **

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier( )

In [ ]:
dtree.fit(XX_train,yy_train)
y_dt = dtree.predict(XX_test)

In [ ]:
print(confusion_matrix(yy_test,y_dt))
print('\n')
print(classification_report(yy_test,y_dt))

** Method 2 : Machine Learning - Random Forest **

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=250)
rfc.fit(XX_train, yy_train)

In [ ]:
y_rfc = rfc.predict(XX_test)

In [ ]:
print(confusion_matrix(yy_test,y_rfc))
print('\n')
print(classification_report(yy_test,y_rfc))

** Method 2 : Deep Learning - full Multi-Layer Perceptron model with full Sessions in Tensorflow **

In [ ]:
import tensorflow as tf
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(XX_train)

In [ ]:
import tensorflow.contrib.learn as learn

In [ ]:
classifier = learn.DNNClassifier(
      feature_columns=feature_columns,
      hidden_units=[40, 20, 10],
      n_classes=5,
      optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))

In [ ]:
classifier.fit(XX_train, yy_train, steps=2500)

In [ ]:
y_tf = list(classifier.predict(XX_test))

In [ ]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(XX_test, yy_test)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

In [ ]:
print(confusion_matrix(yy_test,y_tf))
print('\n')
print(classification_report(yy_test,y_tf))

** Method 4 - GridSearchCV with Random Forest **

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier( verbose=1 )# oob_score = True, )

param_grid = { 
    'n_estimators': [20,50,70,100],# 200, 500,1000],
    'criterion': ['gini', 'entropy'],
    'max_features': [ None, 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(XX_train, yy_train)
print (CV_rfc.best_params_)

In [ ]:
y_grdrfc = CV_rfc.predict(XX_test)

In [ ]:
print(confusion_matrix(yy_test,y_grdrfc))
print('\n')
print(classification_report(yy_test,y_grdrfc))

** Method 4 - Grid Search with Support Vector Machines **

In [ ]:
from sklearn.svm import SVC

In [ ]:
param_grid = {
    'C': [0.1,1, 10, 100, 1000], 
    'gamma': [1,0.1,0.01,0.001,0.0001], 
    'kernel': ['rbf']
} 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)

In [ ]:
# May take awhile!
grid.fit(XX_train,yy_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
y_grdsvc = grid.predict(XX_test)

In [ ]:
print(confusion_matrix(yy_test,y_grdsvc))
print('\n')
print(classification_report(yy_test,y_grdsvc))

## Choice of method
- based on precision random forest seems to be working well
- Random forest shows better predictions for groups of income levels 1 & 2
- the test data will be preprocessed in the same way as the train data
- the test data then will be used for predictions

# Preprocessing the test data

In [ ]:
dftest = pd.read_csv('../input/test.csv')

In [ ]:
dftest.head()

In [ ]:
df7test = dftest.drop(['Id','idhogar','dependency'],axis=1)

In [ ]:
df7test.info()

In [ ]:
len(X_train.columns)

In [ ]:
len(df7test.columns)

- Check for strong correlation and sign of correlation

In [ ]:
for i in list(df7test.drop(['edjefe','edjefa','rez_esc'],axis=1).columns):
    x = np.corrcoef(df7test['SQBdependency'], df7test[i])[0,1]
    if abs(x) > 0.15 : 
        print( i, x )

In [ ]:
df7test['v18q1'] = list(
    map( lambda x,y: ( 0 if y==0 else df7test['v18q1'].mean() ) if x!=x else x,
                    df7test['v18q1'],df7test['v18q'] )
)

In [ ]:
df7test['edjefe'] = df7test['edjefe'].apply(lambda x: int(0) if x=='no'
                                         else int(1) if x=='yes'
                                         else int(x))

In [ ]:
df7test['edjefa'] = df7test['edjefa'].apply(lambda x: int(0) if x=='no'
                                         else int(1) if x=='yes'
                                         else int(x))

In [ ]:
coldf=df7test.columns.values.tolist()
x0 = len( df7test.index )
for i in coldf:
    x = float(len( df7test[pd.isnull(df7test[i])].index )) 
    if x!=0 : print( i, x/x0 * 100 )

** Filling missing values with goal of smooth plotting **

In [ ]:
df8test=df7test.drop(['v2a1','rez_esc'],axis=1).interpolate(method='akima')

coldf=df8test.columns.values.tolist()
x0 = len( df8test.index )
for i in coldf:
    x = float(len( df8test[pd.isnull(df8test[i])].index )) 
    if x!=0 : print( i, x/x0 * 100 )

In [ ]:
df8test.head()

In [ ]:
df8test.info()

In [ ]:
df8test.count()

In [ ]:
X_test = df8test

# Data modeling

** Pipeline - Random Forest**

In [ ]:
import sklearn.pipeline
select = sklearn.preprocessing.StandardScaler()
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=100)
steps = [('feature_selection', select),
        ('random_forest', clf)]

pipeline = sklearn.pipeline.Pipeline(steps)

### fit pipeline on X_train and y_train
pipeline.fit( X_train, y_train )

### call pipeline.predict() on X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )

In [ ]:
plt.figure(figsize=(10,5),facecolor='w') 
# Assign colors for each airline and the names
colors = ['g', 'b']
names = ['Test Prediction', 'Training Prediction']
         
# Make the histogram using a list of lists
# Normalize the flights and assign colors and names
plt.hist([y_prediction,y_train], normed=True,color = colors, label=names, alpha=0.5)

# Plot formatting
plt.legend()
plt.xlabel('Target')
plt.ylabel('Normalized counts')
plt.title('Side-by-Side Histogram with test prediction and training prediction')

In [ ]:
#               TEST HYPOTHESIS
#
testhyp = pd.DataFrame( { 'hhsize':list(X_test['hhsize']),
                       'SQBdependency':list(X_test['SQBdependency']),
                       'Target':list(y_prediction) } )
#data.head()
sns.lmplot(  x='hhsize',y='SQBdependency', hue='Target',
                  fit_reg=False, data=testhyp )

In [ ]:
import sklearn.pipeline
scaler = sklearn.preprocessing.StandardScaler()

### Define RF classifier
rfc = RandomForestClassifier( oob_score = True, verbose=1)


parameters = dict(
                random_forest__n_estimators = [50,100,200,300,400,500,600],
                random_forest__criterion = ['gini', 'entropy'],
                random_forest__max_features = ['sqrt', 'log2']
                )
steps = [('scaler', scaler),
         ('random_forest', rfc)]

pipeline = sklearn.pipeline.Pipeline(steps)

CV_rfc = GridSearchCV(pipeline, param_grid=parameters)

### fit the pipeline on X_train and y_train
#CV_rfc.fit(X_train, y_train)
CV_rfc.fit(X_train.astype(np.float64), 
           y_train.astype(np.float64))

### Print best fitted parameters
print (CV_rfc.best_params_)

### .predict() on the X_test data to make a set of test predictions
y_grdrfc = CV_rfc.predict(X_test)

In [ ]:
plt.figure(figsize=(10,5),facecolor='w') 
# Assign colors for each airline and the names
colors = ['g', 'b']
names = ['Test Prediction', 'Training Prediction']
         
# Make the histogram using a list of lists
# Normalize the flights and assign colors and names
plt.hist([y_grdrfc,y_train], normed=True,color = colors, label=names, alpha=0.5)

# Plot formatting
plt.legend()
plt.xlabel('Target')
plt.ylabel('Normalized counts')
plt.title('Side-by-Side Histogram with test prediction and training prediction')

In [ ]:
#
#               TEST HYPOTHESIS
#
testhyp = pd.DataFrame( { 'hhsize':list(X_test['hhsize']),
                       'SQBdependency':list(X_test['SQBdependency']),
                       'Target':list(y_grdrfc) } )
sns.lmplot( x='hhsize',y='SQBdependency', hue='Target',
            fit_reg=False, data=testhyp)

In [ ]:
plt.figure(figsize=(10,5),facecolor='w') 
# Assign colors for each airline and the names
colors = ['g', 'b']
names = ['RF', 'RF Grid']
         
# Make the histogram using a list of lists
# Normalize the flights and assign colors and names
plt.hist([y_prediction,y_grdrfc], normed=True,color = colors, label=names, alpha=0.5)

# Plot formatting
plt.legend()
plt.xlabel('Target')
plt.ylabel('Normalized counts')
plt.title('Side-by-Side Histogram with test predictions')

# Saving data to a file
- The results from random forest with and without grid are  close 
- The output from RF Grid is saved
- File is checked for consistency 

In [ ]:
for x in (1,2,3,4):
    count  = list(y_grdrfc).count(x)
    print(x,count)

In [ ]:
pd.DataFrame( { 'Id':list(dftest['Id']),
                'Target':list(y_grdrfc.astype(int)) } ).set_index('Id').to_csv('sample_submission.csv', sep=',')

In [ ]:
#
# cross check
#
fpred=pd.read_csv('sample_submission.csv')
plt.hist([fpred['Target'],y_train], normed=True,color = colors, label=names, alpha=0.5)

In [ ]:
testhyp = pd.DataFrame( { 'hhsize':list(X_test['hhsize']),
                       'SQBdependency':list(X_test['SQBdependency']),
                       'Target':list(fpred['Target']) } )
#data.head()
sns.lmplot(  x='hhsize',y='SQBdependency', hue='Target',
                  fit_reg=False, data=testhyp )